## Dataset & Dataloader

Датасет хранит все данные, а даталоудер может по ним итерироваться, управлять созданием батчей, трансформировать данные и т.д. 

In [1]:
!pip install torchmetrics

   ---------------------------------------- 0.0/890.6 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/890.6 kB ? eta -:--:--
   ----------------------------------- ---- 786.4/890.6 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 890.6/890.6 kB 2.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F 
import torch.optim as optim
from torchmetrics import Accuracy
from sklearn.model_selection import train_test_split
import random

def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

Pandas для нас необязателен, но его удобно использовать. 

У нас есть датасет из прошлого семестра про качество вина. Подгрузим его. 

In [3]:
!wget https://raw.githubusercontent.com/rsuh-python/mag2023/main/CL/term02/04-ClassificationTrees/winequalityN.csv

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/rsuh-python/mag2023/main/CL/term02/04-ClassificationTrees/winequalityN.csv')
data.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


В датасете есть пропуски: дропнем их, иначе торчу будет плохо. 

In [7]:
data.dropna(inplace=True)

Для простоты сейчас оставим только числовые данные. 

In [8]:
data.drop('type', axis=1, inplace=True)

Допустим, мы хотим обучить простенькую нейронку на этих данных. Посмотрим распределение классов и их количество:

In [9]:
data.quality.value_counts()

quality
6    2820
5    2128
7    1074
4     214
8     192
3      30
9       5
Name: count, dtype: int64

Давайте укрупним классы: сольем 3, 4 с 5 и 8, 9 с 7 (это, конечно, не дело, но нам пока побаловаться сойдет). 

In [10]:
data.loc[data['quality'] == 8, 'quality'] = 7
data.loc[data['quality'] == 9, 'quality'] = 7
data.loc[data['quality'] == 3, 'quality'] = 5
data.loc[data['quality'] == 4, 'quality'] = 5

Отделим мухи от котлет, нормализуем данные и для красоты перекодируем классы в 0, 1, 2 (хотя в целом пофиг вроде бы). 

In [11]:
X = data.drop('quality', axis=1)
y = data.quality

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y

array([1, 1, 1, ..., 1, 0, 1], dtype=int64)

Для таких простых табличных данных можно использовать стандартные Dataset & DataLoader, но мы посмотрим, как можно написать собственный класс Dataset. 

Поделим на трейн и тест обычным sklearn'ом. 

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

В классе для датасета необходимо перегрузить два метода (помимо init): чтобы экземпляр возвращал свою длину и выдавал пару фичи - ytrue. 

In [29]:
class MyDataset(Dataset):

    def __init__(self, x, y):
        self.x = torch.tensor(x.values, dtype=torch.float32) # если применяем масштабирование (ячейка сверху), то .values надо убрать
        self.y = torch.tensor(y, dtype=torch.long)
 
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

Проверим, как это будет выглядеть:

In [30]:
test = DataLoader(dataset=MyDataset(X_train, y_train), batch_size=5, shuffle=True)

In [19]:
next(iter(test))

[tensor([[-0.6316,  0.4285, -2.1552,  0.1241, -0.2262, -0.5958, -0.8191, -0.3646,
          -0.1635, -0.7367, -0.4041],
         [ 0.2932, -1.0859,  0.8393, -0.8563, -0.3743, -0.3154, -0.1281, -0.5034,
          -0.6652,  1.4705, -0.1518],
         [ 0.3703,  1.4583, -2.1552, -0.7729,  0.9588, -1.0444, -1.7404,  0.3163,
           0.6517,  0.1996,  0.0163],
         [ 2.0658,  1.2159,  0.9090, -0.6686,  0.4552, -0.8201, -1.3506,  1.4466,
           1.0906,  0.6010, -0.9085],
         [ 0.3703, -0.1773, -0.6231, -0.7311,  0.3959, -0.0911,  0.3148, -0.2985,
          -0.6652,  0.6010, -0.5722]]),
 tensor([0, 1, 2, 0, 0])]

Теперь можно собрать трейн и тест. На трейне хотим шаффлить, чтобы было как можно больше вариаций в батчах, а на тесте скорее нет - для детерминированности результата. 

In [32]:
train_loader = DataLoader(dataset=MyDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=MyDataset(X_test, y_test), batch_size=BATCH_SIZE, shuffle=False)

Зададим параметры и напишем класс с моделью. 

In [31]:
INPUT_SIZE = 11
HIDDEN_SIZE = 35
OUTPUT_SIZE = 3
LEARNING_RATE = 1e-3
EPOCHS = 100
BATCH_SIZE = 1024 # у нас очень маленький датасет с маленьким набором фич, можем хоть весь целиком в батч запихнуть

In [33]:
class TorchModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=INPUT_SIZE, out_features=HIDDEN_SIZE) # линейный полносвязный слой
        self.fc2 = nn.Linear(in_features=HIDDEN_SIZE, out_features=HIDDEN_SIZE) # скрытый слой
        self.out = nn.Linear(in_features=HIDDEN_SIZE, out_features=OUTPUT_SIZE) # выходной слой

    def forward(self, x):
        x = nn.LeakyReLU()(self.fc1(x)) # побалуемся с функциями активации
        x = nn.LeakyReLU()(self.fc2(x))
        x = self.out(x)
        return x

In [26]:
# проверяем что всё у нас правильно работает
batch = next(iter(train_loader)) # возвращает х и у

In [34]:
batch

[tensor([[-1.0940e+00, -7.2249e-01, -4.8383e-01,  ...,  9.6521e-01,
           3.3342e-01, -6.7750e-02],
         [-1.7106e+00,  2.4275e+00, -2.2248e+00,  ...,  4.9785e+00,
           4.0030e-01,  1.6979e+00],
         [ 3.7028e-01,  6.4999e-02,  1.1875e+00,  ..., -4.7708e-01,
          -1.0043e+00,  1.6138e+00],
         ...,
         [-4.0041e-01,  4.4231e-03,  7.6969e-01,  ...,  8.7294e-02,
          -6.0297e-01,  1.7820e+00],
         [-1.4023e+00, -6.6192e-01,  2.8221e-01,  ...,  1.5000e-01,
           3.0757e+00,  1.3616e+00],
         [-3.2334e-01,  7.9191e-01, -1.6677e+00,  ...,  5.2625e-01,
           8.0161e-01, -7.4038e-01]]),
 tensor([1, 1, 2,  ..., 1, 2, 1])]

Соберем нужные штуки и инициализируем модель

In [27]:
criterion = nn.CrossEntropyLoss()
accuracy = Accuracy(task='multiclass', num_classes=3)

set_random_seed(42)
model = TorchModel() # инициализируем нашу модель
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [37]:
features, labels = batch

model(batch[0])

tensor([[0.1706, 0.0956, 0.0769],
        [0.2888, 0.1575, 0.1211],
        [0.0767, 0.0846, 0.2086],
        ...,
        [0.1189, 0.1210, 0.1300],
        [0.3533, 0.1206, 0.3286],
        [0.1198, 0.0625, 0.1126]], grad_fn=<AddmmBackward0>)

Скопипастим из прошлых тетрадок трейнлуп...

In [38]:
train_loss_values = []
train_accuracy_values = []
valid_loss_values = []
valid_accuracy = []

def run_train():
    step = 0
    for epoch in range(EPOCHS):
        running_loss = []
        running_acc = []
        for features, label in train_loader:
            # Reset gradients

            output = model(features)
            # Calculate error and backpropagate
            loss = criterion(output, label) # функция потерь
            loss.backward()
            acc = accuracy(output, label).item()

            # Update weights with gradients
            optimizer.step()
            optimizer.zero_grad()

            running_loss.append(loss.item())
            running_acc.append(acc)

        train_loss_values.append(np.mean(running_loss))
        train_accuracy_values.append(np.mean(running_acc))
        if epoch % 20 == 0:
            print(f'EPOCH {epoch}: train_loss: {train_loss_values[-1]}')# train_accuracy_values[-1]))


        # Run validation
        running_loss = []
        running_acc = []
        for features, label in test_loader:
            output = model(features)
            # Calculate error and backpropagate
            loss = criterion(output, label)
            acc = accuracy(output, label).item()

            running_loss.append(loss.item())
            running_acc.append(acc)

        valid_loss_values.append(np.mean(running_loss))
        valid_accuracy.append(np.mean(running_acc))
        if epoch % 20 == 0:
            print(f'EPOCH {epoch}: valid_loss: {valid_loss_values[-1]}, valid_accuracy: {valid_accuracy[-1]}')
        
    return train_loss_values, train_accuracy_values, valid_loss_values, valid_accuracy

In [39]:
train_loss_values, train_accuracy_values, valid_loss_values, valid_accuracy = run_train()

EPOCH 0: train_loss: 2.9561476508776345
EPOCH 0: valid_loss: 1.7319461703300476, valid_accuracy: 0.23183012753725052
EPOCH 20: train_loss: 1.0266366203625996
EPOCH 20: valid_loss: 1.0082798898220062, valid_accuracy: 0.4625275880098343
EPOCH 40: train_loss: 1.009975751241048
EPOCH 40: valid_loss: 0.9958680272102356, valid_accuracy: 0.4762938618659973
EPOCH 60: train_loss: 0.985081801811854
EPOCH 60: valid_loss: 0.978121429681778, valid_accuracy: 0.5026937127113342
EPOCH 80: train_loss: 0.9581185380617777
EPOCH 80: valid_loss: 0.9524401724338531, valid_accuracy: 0.4893050044775009
